# Test Dataset to check

In [1]:
import multiprocessing
NUM_PROCESSORS=multiprocessing.cpu_count()
# print("Cpu count: ",NUM_PROCESSORS)

In [2]:
#as it turned out interactive shell (like Jupyter cannot handle CPU multiprocessing well so check which medium the code is runing)
#we will write code in Jupyter for understanding purposes but final execuation will be in shell
from ipynb.fs.full.Utils import isnotebook
from ipynb.fs.full.Dataset import get_data
import networkx as nx
from torch_geometric.utils import to_networkx, from_networkx
import torch_geometric.utils.homophily as homophily
import copy
import ipynb.fs.full.utils.MoonGraph as MoonGraph

In [3]:
import torch
import torch.nn as nn
from torch_sparse import SparseTensor
from tqdm import tqdm
import math
import time
import os

import random
from random import choice
random.seed(12345)
import numpy as np
np.random.seed(12345)

In [4]:
import sklearn
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from multiprocessing.pool import ThreadPool, Pool

In [5]:
#data, dataset = get_data('Cora')

In [6]:
from scipy.sparse import identity
from scipy.sparse import csgraph
from scipy import linalg
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from scipy import sparse, stats
#from scipy.sparse import linalg

## Effective Resistance (True)

In [7]:
def ER(u,v, L_inv,N):
    
    x_u = np.zeros((N,))
    x_v = np.zeros((N,)) 
    x_u[u] = 1
    x_v[v] = 1
    
    d_uv=x_u-x_v
    
    R_uv=d_uv.dot(L_inv.dot(d_uv)) ## (x_u-x_v)^T*L'*(x_u-x_v)
    
    return R_uv

def compute_ER(Adj, L_inv):

    start_nodes, end_nodes, weights = sparse.find(sparse.tril(Adj))
    n = np.shape(Adj)[0]
    Reff = sparse.lil_matrix((n,n))
    
    for orig, end in zip(start_nodes, end_nodes):
        Reff[orig,end] = ER(orig, end, L_inv, n)
        
    return Reff

def EffectiveResistance(data):
    
    N = data.num_nodes
    E = data.num_edges
    
    Adj = coo_matrix(([1] * E, (data.edge_index[0], data.edge_index[1])), shape=(N,N))    

    #Adj = nx.adjacency_matrix(G)
    L, D  = csgraph.laplacian(Adj, normed=False, return_diag=True)
    
    #print(np.allclose(L.todense(), np.diag(D)-Adj)) #verify L=D-A
        
    L_inv = linalg.pinv(L.todense())
    
    Reff=compute_ER(Adj, L_inv)
    
    return Reff    

# Reff = EffectiveResistance(data)
# Reff

## Effective Resistance (Local Approx)

In [8]:
class EffectiveRessistance():
    
    def __init__(self, data, eps=0.9, lmbda=0.10):
        
        self.N = N = data.num_nodes
        self.E = E = data.num_edges
        self.data = data
        self.eps = eps
        self.lmbda = lmbda

#         self.Adj = SparseTensor(
#             row=data.edge_index[0], 
#             col=data.edge_index[1],
#             value=torch.arange(E, device=data.edge_index.device),sparse_sizes=(N, N))
        
        if N == 232965:
            G_fillename="reddit.gpickle"
            if os.path.exists(G_fillename)==False:
                print("Graph is not found, creating it....")
                self.G = to_networkx(data, to_undirected=True)
                nx.write_gpickle(self.G, G_fillename)
                print("Done")
            else:
                print("Loading Saved graph...")
                self.G = nx.read_gpickle(G_fillename)
                print("Done")
    
        else:
            self.G = to_networkx(data, to_undirected=True)
    
    def random_walk(self, s, l):
        v = s;
        for i in range(l):  
            if (len(self.G[v]) == 0):
                continue;
            v = choice(list(self.G.neighbors(v)))

        return v


    def er_edge(self, index, s, t, eps=0.1, lmbda=0.1):
        
#         l = math.ceil(math.log(4 / (eps * (1 - lmbda))) / math.log(1.0 / lmbda) / 2)
#         r = int(math.ceil(40 * l * l * math.log(80 * l) / (eps * eps)))

        l = 4
        r = 100
#         print(l,r)     
        
        delta = 0

        
        
        s_deg = self.G.degree[s]
        t_deg = self.G.degree[t]
        
        #print(self.G.degree[s], self.G.degree[t])

        for i in range(l):
            Xis = 0; Xit = 0; Yis = 0; Yit = 0;

            for j in range(r):
                v = self.random_walk(s, i)
                if (v == s):
                    Xis+=1
                if (v == t):
                    Xit+=1    

            for j in range(r):
                v = self.random_walk(t, i);
                if (v == s):
                    Yis+=1;
                if (v == t):
                    Yit+=1;

            deltai = float(Xis) / s_deg  - float(Xit) / t_deg - float(Yis) / s_deg + float(Yit) / t_deg
            deltai /= r;
            delta += deltai;

        return index, max(0,delta)
    
    def er_weight(self):
        
        weight = np.zeros(len(self.data.edge_index[0]))
        
        pbar = tqdm(total=self.E)
        pbar.set_description(f'Edges')
        
        for i, edge in enumerate(self.data.edge_index.T.tolist()):
            _, weight[i] = self.er_edge(i, edge[0], edge[1], eps=self.eps, lmbda=self.lmbda)
            
            pbar.update(1)
            
        pbar.close()
            
        weight = torch.Tensor(weight)
            
        return weight
    
    
    def block(self, edge_list):
        
        for (i, u, v) in edge_list:
            _, self.weight[i] = self.er_edge(i, u, v, eps=self.eps, lmbda=self.lmbda)        
        
        return len(edge_list)
    
    def er_weight_parallel(self):
        
        pool_size = NUM_PROCESSORS        
        #pool_size=1
        print("Pool Size: ", pool_size)        
        pool = Pool(pool_size)
        
        elem_size= 1000000
        num_blocks = int(self.E/elem_size)
                            
        self.weight = np.zeros(len(data.edge_index[0]))
        u =  self.data.edge_index[0].tolist()
        v =  self.data.edge_index[1].tolist()
        
        param = list(zip(range(self.E), u, v))
        params = [ param[i*elem_size:(i+1)*elem_size] for i in range(num_blocks)]
        
        if num_blocks*elem_size<self.E:
            params.append(param[num_blocks*elem_size:self.E]) 
                          
        #print(params)
        
        pbar = tqdm(total=self.E)
        pbar.set_description(f'Edges') 
        
        for num_el in pool.imap_unordered(self.block, params):            
            pbar.update(num_el)        
        pbar.close()
                
        return self.weight

    def compute_weights(self):
        if isnotebook():
            weight = self.er_weight()    
        else:
            weight = self.er_weight_parallel()
        
        return weight
    
# er = EffectiveRessistance(data, eps=0.1, lmbda=0.1)
# er.er_edge(0, 20, 21)
#er.er_weight()
# #er.er_weight_parallel()
#weight  = er.compute_weights()
# torch.save(weight, 'Weights/reddit_er.pt')
# torch.load('Weights/reddit_er.pt')

In [9]:
def CombineNNandER(data, metric='cosine'):
    
    from ipynb.fs.full.KNNWeights import KNNWeight
    
    NN = KNNWeight(data, metric=metric)
    ER = EffectiveRessistance(data, eps=0.9, lmbda=0.10)
    
    nn_weight = NN.compute_weights()
    er_weight = ER.er_weight()
    
    return nn_weight*er_weight

#CombineNNandER(data, metric='cosine')

# Main

In [10]:
if __name__ == '__main__':  
    
    data, dataset = get_data('Cora')
    
#     print("Node Homophily:", homophily(data.edge_index, data.y, method='node'))
#     print("Edge Homophily:", homophily(data.edge_index, data.y, method='edge'))
#     print("Edge_insensitive Homophily:", homophily(data.edge_index, data.y, method='edge_insensitive'))    
    
#     er = EffectiveRessistance(data)

    start = time.time()    
    #data.weight  = er.er_weight(eps=0.9, lmbda=0.1)
    data.weight = CombineNNandER(data, metric='cosine')
    end = time.time()
    print("Execution time: ", end-start)

    if 'weight' in data:
        cp_data= copy.deepcopy(data)
        G = to_networkx(cp_data, to_undirected=True, edge_attrs=['weight'])
        to_remove = [(a,b) for a, b, attrs in G.edges(data=True) if attrs["weight"] <0.5 ]
        G.remove_edges_from(to_remove)
        updated_data = from_networkx(G)

        print("Node Homophily:", homophily(updated_data.edge_index, cp_data.y, method='node'))
        print("Edge Homophily:", homophily(updated_data.edge_index, cp_data.y, method='edge'))
        print("Edge_insensitive Homophily:", homophily(updated_data.edge_index, cp_data.y, method='edge_insensitive'))    
        
    
    None

Data directory:  /scratch/gilbreth/das90/Dataset/
Result directory: /scratch/gilbreth/das90/Dataset/RESULTS/

Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True
Node Homophily: 0.825157880783081
Edge Homophily: 0.8099659085273743
Edge_insensitive Homophily: 0.7657181620597839
Assortativity:  -0.06587088108062744
Metric:  cosine


Edges: 100%|██████████| 10556/10556 [00:16<00:00, 649.65it/s]

Execution time:  17.452056884765625
Node Homophily: 0.3830185830593109
Edge Homophily: 0.8399452567100525
Edge_insensitive Homophily: 0.7945449948310852
